In [1]:
import os
import keras

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import librosa
import librosa.display
import re
from sklearn.model_selection import train_test_split

In [3]:
ds_dir = '/arch/mnist_data'
filenames = []

for root, dirs, files in os.walk(ds_dir):
    for file in files:
        filenames.append(os.path.join(root,file))
filenames = [re.sub(r'(\\)', '/', name) for name in filenames[1:]]

samples_number = len(filenames)
print('Total number of samples:', samples_number)

Total number of samples: 30000


In [4]:
labels = np.array(["0","1","2","3","4","5","6","7","8","9"])
train_files,test_files = train_test_split(filenames, test_size=0.3, shuffle=True)

print('Training data size', len(train_files))
print('Test data size', len(test_files))

Training data size 21000
Test data size 9000


In [8]:
def get_label(file_path):
    label = re.findall(r'/([0-9])_', file_path)[0]
    label = np.argmax(label == labels)
    return label

In [10]:
input_shape = (128,44)

In [7]:
def get_ms(file_path):
    y, sr = librosa.load(file_path)
    ms = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
    ms = np.resize(ms, input_shape)
    
    ms = np.log(ms + 1e-9)
    ms = librosa.util.normalize(ms)
    return ms.ravel()

In [8]:
def preprocess_dataset(files):
    output_labels = np.array([get_label(file) for file in files])
    output_ms = np.array([get_ms(file) for file in files])
    return output_ms, output_labels

In [25]:
train_ms,train_labels = preprocess_dataset(train_files)
test_ms,test_labels = preprocess_dataset(test_files)

KeyboardInterrupt: 

In [9]:
from sklearn.model_selection import GridSearchCV

In [49]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Bayes

In [64]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()
nbc.fit(train_ms,train_labels)
test_pred = nbc.predict(test_ms)

In [65]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.5246666666666666 0.5603083326312298 0.5275950401581319 0.523091491691517


# K Neighbors

In [23]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = [{"n_neighbors": [i for i in range(1, 50)]}]

knn_reg = KNeighborsClassifier()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)

grid_search.fit(train_ms,train_labels)
p_knn = grid_search.best_params_
p_knn

Fitting 5 folds for each of 49 candidates, totalling 245 fits


{'n_neighbors': 1}

In [67]:
kNN_clf = KNeighborsClassifier(n_neighbors=p_knn['n_neighbors'])
kNN_clf.fit(train_ms,train_labels)
test_pred = kNN_clf.predict(test_ms)

In [68]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.9615555555555556 0.9618582496696565 0.9617385019484525 0.961709563659569


# SVC

In [11]:
from sklearn.svm import SVC
param_grid = [{"C": [i for i in np.arange(0.1, 10, 0.1)],
              "kernel": ['linear', 'poly', 'rbf', 'sigmoid']}]

svc = SVC()
grid_search = GridSearchCV(svc, param_grid, cv=2, verbose=3, n_jobs=2)

grid_search.fit(train_ms,train_labels)
p_svc = grid_search.best_params_
p_svc

Fitting 2 folds for each of 396 candidates, totalling 792 fits


{'C': 7.0, 'kernel': 'rbf'}

In [69]:
svc_clf = SVC(C=p_svc['C'], kernel=p_svc['kernel'])
svc_clf.fit(train_ms,train_labels)
test_pred = svc_clf.predict(test_ms)

In [70]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.9827777777777778 0.9828218063508887 0.9828538093646163 0.9828169433344973


# NuSVC

In [10]:
from sklearn.svm import NuSVC
param_grid = [{"nu": [i for i in np.arange(0, 1, 0.1)],
              "kernel": ['linear', 'poly', 'rbf', 'sigmoid']}]

nu_svc = NuSVC()
grid_search = GridSearchCV(nu_svc, param_grid, cv=2, verbose=3, n_jobs=4)

grid_search.fit(train_ms,train_labels)
p_nu_svc = grid_search.best_params_
p_nu_svc

Fitting 2 folds for each of 40 candidates, totalling 80 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
8 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "D:\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 189, in sklearn.svm._libsvm.fit
ValueError: 

{'kernel': 'rbf', 'nu': 0.1}

In [71]:
nu_svc_clf = NuSVC(kernel=p_nu_svc['kernel'], nu=p_nu_svc['nu'])
nu_svc_clf.fit(train_ms,train_labels)
test_pred = nu_svc_clf.predict(test_ms)

In [72]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.9781111111111112 0.9782100283875357 0.9781793861673531 0.9781536982578132


# LinearSVC

In [10]:
from sklearn.svm import LinearSVC
param_grid = [{"C": [i for i in np.arange(0.5, 10, 0.5)],
              "penalty": ['l1', 'l2'],
              "dual": [True, False],
              "multi_class": ['ovr', 'crammer_singer']}]

lin_svc = LinearSVC()
grid_search = GridSearchCV(lin_svc, param_grid, cv=2, verbose=3, n_jobs=4)

grid_search.fit(train_ms,train_labels)
p_lin_svc = grid_search.best_params_
p_lin_svc

Fitting 2 folds for each of 152 candidates, totalling 304 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
38 fits failed out of a total of 304.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
38 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "D:\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "D:\anacon

{'C': 0.5, 'dual': True, 'multi_class': 'crammer_singer', 'penalty': 'l1'}

In [73]:
lin_svc_clf = LinearSVC(C=p_lin_svc['C'], dual=p_lin_svc['dual'], 
                        multi_class=p_lin_svc['multi_class'], penalty=p_lin_svc['penalty'])
lin_svc_clf.fit(train_ms,train_labels)
test_pred = lin_svc_clf.predict(test_ms)

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [74]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.9592222222222222 0.959332255263876 0.9593479165894104 0.9593073708577251


# Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

param_grid = [{"n_estimators": [i for i in np.arange(100, 500, 10)], "max_depth": [i for i in range(10, 50)]}]

rfc_reg = RandomForestClassifier()
grid_search = GridSearchCV(rfc_reg, param_grid, cv=2, n_jobs=4, verbose=3)

grid_search.fit(train_ms,train_labels)
p_rfc = grid_search.best_params_
p_rfc

Fitting 2 folds for each of 1600 candidates, totalling 3200 fits


{'max_depth': 46, 'n_estimators': 420}

In [75]:
rfc_clf = RandomForestClassifier(max_depth=p_rfc['max_depth'], n_estimators=p_rfc['n_estimators'])
rfc_clf.fit(train_ms,train_labels)
test_pred = rfc_clf.predict(test_ms)

In [76]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.9336666666666666 0.9338839242456158 0.9339832706766187 0.9337511978541275


# ExtraTrees

In [18]:
from sklearn.ensemble import ExtraTreesClassifier

param_grid = [{"n_estimators": [i for i in np.arange(300, 500, 10)], 
               "criterion": ['gini', 'entropy', 'log_loss'],
               "max_depth": [i for i in np.arange(30, 55, 5)],
              "max_leaf_nodes": [i for i in np.arange(10, 32, 2)],
              "bootstrap": [True, False],
              "oob_score": [True, False]}]

etc_reg = ExtraTreesClassifier()
grid_search = GridSearchCV(etc_reg, param_grid, cv=2, n_jobs=4, verbose=3)

grid_search.fit(train_ms,train_labels)
p_etc = grid_search.best_params_
p_etc

Fitting 2 folds for each of 13200 candidates, totalling 26400 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
13200 fits failed out of a total of 26400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4400 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "D:\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
 

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 45,
 'max_leaf_nodes': 30,
 'n_estimators': 450,
 'oob_score': True}

In [77]:
etc_clf = ExtraTreesClassifier(max_depth=p_etc['max_depth'], n_estimators=p_etc['n_estimators'], 
                               criterion=p_etc['criterion'], max_leaf_nodes=p_etc['max_leaf_nodes'], 
                               bootstrap=p_etc['bootstrap'], oob_score=p_etc['oob_score'])
etc_clf.fit(train_ms,train_labels)
test_pred = etc_clf.predict(test_ms)

In [78]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.6897777777777778 0.7141300286305049 0.6930224757393411 0.6879189084097986


# DecisionTree

In [19]:
from sklearn.tree import DecisionTreeClassifier

param_grid = [{"criterion": ['gini', 'entropy', 'log_loss'],
               "max_depth": [i for i in np.arange(30, 55, 5)],
              "max_leaf_nodes": [i for i in np.arange(10, 32, 2)]}]

dtc = DecisionTreeClassifier()
grid_search = GridSearchCV(dtc, param_grid, cv=2, n_jobs=4, verbose=3)

grid_search.fit(train_ms,train_labels)
p_dtc = grid_search.best_params_
p_dtc

Fitting 2 folds for each of 165 candidates, totalling 330 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
110 fits failed out of a total of 330.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "D:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_message, FitFailedWarning)
D:\anaconda3\lib\sit

{'criterion': 'gini', 'max_depth': 30, 'max_leaf_nodes': 30}

In [79]:
dtc_clf = DecisionTreeClassifier(max_depth=p_dtc['max_depth'], max_leaf_nodes=p_dtc['max_leaf_nodes'], 
                                 criterion=p_dtc['criterion'])
dtc_clf.fit(train_ms,train_labels)
test_pred = dtc_clf.predict(test_ms)

In [80]:
print(accuracy_score(test_labels, test_pred),
precision_score(test_labels, test_pred, average='macro'),
recall_score(test_labels, test_pred, average='macro'),
f1_score(test_labels, test_pred, average='macro'))

0.4172222222222222 0.4513284994340073 0.4175026596678477 0.42658331033426167


# LSTM

In [5]:
def get_ms_lstm(file_path):
    y, sr = librosa.load(file_path)
    ms = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
    ms = np.resize(ms, input_shape)
    
    ms = np.log(ms + 1e-9)
    ms = librosa.util.normalize(ms)
    return ms

In [6]:
def preprocess_dataset_lstm(files):
    output_labels = np.array([get_label(file) for file in files])
    output_ms = np.array([get_ms_lstm(file) for file in files])
    return output_ms, output_labels

In [27]:
train_ms2,train_labels2 = preprocess_dataset_lstm(train_files)
test_ms2,test_labels2 = preprocess_dataset_lstm(test_files)

In [40]:
epoch=25

In [56]:
model = keras.Sequential()
model.add(keras.layers.LSTM(units=256, dropout=0.1, 
                            return_sequences=True, 
                            input_shape=(128,44)))
model.add(keras.layers.LSTM(units=128,  dropout=0.1, 
                            return_sequences=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10))

In [57]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, decay=1e-2/epoch),
    metrics=["accuracy"],
)

In [58]:
model.fit(
    train_ms2,train_labels2, 
    validation_data=(test_ms2,test_labels2), 
    batch_size=64, 
    epochs=epoch,
    shuffle=True)

Epoch 1/25
329/329 [==============================] - 220s 657ms/step - loss: 2.0779 - accuracy: 0.2364 - val_loss: 2.0106 - val_accuracy: 0.2218
Epoch 2/25
329/329 [==============================] - 218s 664ms/step - loss: 1.5899 - accuracy: 0.3936 - val_loss: 1.5931 - val_accuracy: 0.3931
Epoch 3/25
329/329 [==============================] - 229s 696ms/step - loss: 1.2502 - accuracy: 0.5282 - val_loss: 1.3189 - val_accuracy: 0.5004
Epoch 4/25
329/329 [==============================] - 231s 703ms/step - loss: 0.9584 - accuracy: 0.6461 - val_loss: 0.9508 - val_accuracy: 0.6497
Epoch 5/25
329/329 [==============================] - 233s 707ms/step - loss: 0.8109 - accuracy: 0.6964 - val_loss: 1.1405 - val_accuracy: 0.6071
Epoch 6/25
329/329 [==============================] - 234s 713ms/step - loss: 0.6941 - accuracy: 0.7441 - val_loss: 0.8902 - val_accuracy: 0.6891
Epoch 7/25
329/329 [==============================] - 236s 717ms/step - loss: 0.5854 - accuracy: 0.7807 - val_loss: 0.7754 -

In [59]:
test_pred = model.predict(test_ms2)

282/282 [==============================] - 40s 139ms/step


In [60]:
test_pred2 = [np.argmax(x) for x in test_pred]

In [61]:
print(accuracy_score(test_labels2, test_pred2),
precision_score(test_labels2, test_pred2, average='macro'),
recall_score(test_labels2, test_pred2, average='macro'),
f1_score(test_labels2, test_pred2, average='macro'))

0.934 0.9360419003610723 0.9344363414504109 0.9338980411266476
